In [ ]:
!pip install selenium
!pip install webdriver_manager
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [92]:
## All the external libraries
import re
import csv
from getpass import getpass
from time import sleep
import random
import datetime as dt
from datetime import datetime


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
#from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver import Chrome

In [ ]:
## All the internal libraries

In [148]:
####### All the functions #######

## Get random time offset
def get_rand_offset(t=4):
    r = random.randint(0, t)
    r += random.random()
    return r

#### Get the browser
## Return <class 'selenium.webdriver.chrome.webdriver.WebDriver'>
## Opens an empty browser
def get_driver():
    if driver == None:
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      return  webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    else:
        return driver

## Go to address
def visit_site(site="https://www.google.com"):
    driver.get(site)

    
## Add tweets to file
def write_to_file(fname, data):
    f = open(fname, "a")
    f.write('\n'.join(data))
    f.close()

## Get n tweets
def get_ntweets(num_tweets=100):
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        page_cards = []
        while len(page_cards) < 1:
            page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

        min_tweets = -1*min(len(page_cards), 15)

        for card in page_cards[min_tweets:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    if len(data) == num_tweets:
                        scrolling = False
                        break
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1+get_rand_offset(2))
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(1+get_rand_offset(2)) # attempt another scroll
            else:
                last_position = curr_position
                break
    return data
    
##get the date for the next day
def nextDay(date):
    end_date = date + dt.timedelta(days=1)
    return end_date

## Get specific day tweets
def get_specific_date_tweets(search_term, date, num_tweets, filename):
    #'https://twitter.com/search?f=live&q=(%23Covid19)%20lang%3Aen%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query')
    url_to_visit = 'https://twitter.com/search?f=live&q=(%23'+search_term+')%20lang%3Aen%20until%3A'+(nextDay(date)).strftime('%Y-%m-%d')+'%20since%3A'+date.strftime('%Y-%m-%d')+'&src=typed_query'
    print(url_to_visit)
    visit_site(site=url_to_visit)
    tweets = get_ntweets(num_tweets=num_tweets)
    write_to_file(fname=filename, data=tweets)
    pass

    

## Description
## Takes
## Returns
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
    the emoji character."""
    emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt)
    return tweet   

In [149]:
## Initialization: To be run only once at the begining
driver = None

In [150]:
## Get the browser
driver = get_driver()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [ ]:
## Webdriver support following
print(f'dir of webdriver: {dir(driver)}')

In [152]:
## Visit twitter & Login to Twitter
#username='ishaanspotify'
#password='dummyaccount'
visit_site(site="https://twitter.com/i/flow/login")
#driver.find_element(by=By.NAME, value="Phone, email address, or username").click()
#driver.find_element(by=By.XPATH, value='/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[5]/label/div/div[2]/div/input').send_keys(username)
#driver.find_element(by=By.XPATH, value='//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[6]/div').click()
#driver.find_element(by=By.XPATH, value='//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input').send_keys(password)


In [154]:
twitter_cookies = driver.get_cookies()

In [155]:
print(twitter_cookies)

[{'domain': '.twitter.com', 'expiry': 1709923828, 'httpOnly': False, 'name': '_ga', 'path': '/', 'secure': False, 'value': 'GA1.2.112168057.1646851828'}, {'domain': '.twitter.com', 'expiry': 1646851888, 'httpOnly': False, 'name': '_gat', 'path': '/', 'secure': False, 'value': '1'}, {'domain': '.twitter.com', 'expiry': 1646938228, 'httpOnly': True, 'name': 'att', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '1-EU4dYVholKnOi9RxPKrkPUtZmogeFoODqJjBceQK'}, {'domain': '.twitter.com', 'expiry': 1646862627, 'httpOnly': False, 'name': 'gt', 'path': '/', 'secure': True, 'value': '1501631517352673280'}, {'domain': '.twitter.com', 'expiry': 1646938228, 'httpOnly': False, 'name': '_gid', 'path': '/', 'secure': False, 'value': 'GA1.2.236837195.1646851828'}, {'domain': '.twitter.com', 'expiry': 1709923826, 'httpOnly': False, 'name': 'guest_id', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'v1%3A164685182643480238'}, {'domain': '.twitter.com', 'expiry': 1709923826, 'httpOnly

In [145]:
## Deletes the current cookies
driver.delete_all_cookies()

In [136]:
## Restore the cookie
for cookie in twitter_cookies:
    #print(f'cookie:{cookie}')
    driver.add_cookie(cookie_dict=cookie)

In [137]:
## Refreshes the page
driver.refresh()

In [156]:
## Search
visit_site(site='https://twitter.com/search?q=(%23Covid19)%20lang%3Aen%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query')

In [ ]:
# get all tweets on the page
data = get_ntweets(num_tweets=5)

In [157]:
##get tweets for a looping date
startdate='01-01-2020'
enddate='01-03-2020'
start_date = datetime.strptime(startdate,"%m-%d-%Y")
end_date = datetime.strptime(enddate, "%m-%d-%Y")
date=start_date
while(date<end_date):
    get_specific_date_tweets('covid19',date,10, 'test1.txt')
    date = date + dt.timedelta(days=1)
    


https://twitter.com/search?q=(%23covid19)%20lang%3Aen%20until%3A2020-01-02%20since%3A2020-01-01&src=typed_query


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


KeyboardInterrupt: ignored

In [ ]:
print(data)

In [ ]:
len(data)